# Applying CLAY filtering and getting layout embeddings
**21/03/2022**

**Roberto Araya Day**

First, we apply CLAY label flags to the tree view hierarchy in the json files and remove the invalid nodes. Then, we train clustering algorithms with the changed view hierarchies from the screenshots of the most popular apps.

In [1]:
import os
import argparse
import json
import numpy as np
import torch
from s2v_code.Screen2Vec import Screen2Vec
from sentence_transformers import SentenceTransformer
from s2v_code.prediction import TracePredictor
from s2v_code.autoencoder import ScreenLayout, LayoutAutoEncoder
from s2v_code.UI_embedding.UI2Vec import HiddenLabelPredictorModel, UI2Vec
from s2v_code.dataset.playstore_scraper import get_app_description
from s2v_code.dataset.rico_utils import get_all_labeled_uis_from_rico_screen, ScreenInfo
from s2v_code.dataset.rico_dao import load_rico_screen_dict

import pandas
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.cluster import AgglomerativeClustering, DBSCAN, KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import PCA
from datetime import datetime
from itertools import compress
import random

from google_play_scraper import app
import csv
import requests
import pandas as pd
from pathlib import Path
import umap.umap_ as umap


**Important Paths**

In [2]:
# path to important folders (may need to be changed)
screenvec = os.path.join(os.getcwd())
rico_dataset = os.path.join(os.path.dirname(os.getcwd()), \
"RICO_CLAY_data", "unique_uis.tar", "unique_uis", "combined")
rico_layout_dataset = os.path.join(os.path.dirname(os.getcwd()), \
"RICO_CLAY_data", "sem_annot", "semantic_annotations")

pretrained_model = os.path.join(screenvec, "s2v_pre-trained_model")
saved_tensors = os.path.join(os.getcwd(), "rico_tensors")
saved_popular_tensors = os.path.join(os.getcwd(), "popular_rico_tensors")

# path of the apps metadata
ui_details_csv = os.path.join(os.path.dirname(os.getcwd()), \
"RICO_CLAY_data", "ui_details.csv")
app_details_csv = os.path.join(os.path.dirname(os.getcwd()), \
"RICO_CLAY_data", "app_details.csv")

ui_names_csv = os.path.join(os.path.dirname(os.getcwd()), \
"RICO_CLAY_data", "ui_layout_vectors", "ui_layout_vectors", "ui_names.json")
layout_embedding_csv = os.path.join(os.path.dirname(os.getcwd()), \
"RICO_CLAY_data", "ui_layout_vectors", "ui_layout_vectors", "ui_vectors.npy")
clay_labels_csv = os.path.join(os.path.dirname(os.getcwd()), \
"RICO_CLAY_data", "clay_labels.csv")
clay_label_map = os.path.join(os.path.dirname(os.getcwd()), \
"RICO_CLAY_data", "label_map.txt")


# path of pre-trained model
gui_element_path = os.path.join(pretrained_model, "UI2Vec_model.ep120")
screen2vec_path = os.path.join(pretrained_model,"Screen2Vec_model_v4.ep120")
layout_encoder = os.path.join(pretrained_model,"layout_encoder.ep800")

# path of the saved plots and figures
saved_plt_path = os.path.join(os.getcwd(), "results")
cluster_examples = os.path.join(saved_plt_path, "cluster_examples")

# path for saving the results
records = "records.txt"
records_path = os.path.join(saved_plt_path, records)

In [21]:
with open(clay_labels_csv) as csvfile:
    # maps the information in each row of the screenshot details to a dict
    reader = csv.DictReader(csvfile)
    counter = 0
    
    # iterate for each detail of screenshot 
    for row in reader:
        counter +=1
        screen_id, node_id, label = row.values()
        # get the app's name from the screenshot
        print(screen_id, node_id, label)
        
        screen_file = os.path.join(rico_dataset, f"{screen_id}.json")
        with open(screen_file) as f:
            rico_screen = json.load(f)
        #print(rico_screen["activity"]["root"]["children"])
        if counter == 5: break

25845 b6e43f6 -1
25845 eebc03b 0
25845 7950358 0
25845 4c03b1d 0
25845 c55d4de 1


## 1. Process RICO dataset using CLAY pipeline

We use the code from the repository and:
1. Read the files of the CLAY repository *clay_labels* and *label_map*.
2. Filter the invalid nodes and assign new labels.
3. Save the layout information with the new labels and boundary boxes of the components in '.xml' files.

In [31]:
#https://gist.github.com/dusskapark/c59eae6cae2b822bf7d133a2860c4c51#file-convert_rico_for_object_detection_based_on_clay-ipynb

import os
import json
from xml.etree.ElementTree import Element, SubElement
import csv
import sys

sys.setrecursionlimit(10**6)


def beautify(elem, indent=0):
    """
    Converts the XML tree to a string.

    :param elem: xml element
    :param indent: Indent Level to display in front
    """
    result0 = f"{'    ' * indent}<{elem.tag}>"
    
    # If there is a value in the tag, immediately print the value
    if elem.text is not None:
        result0 += elem.text + f"</{elem.tag}>\n"
    # 값이 없고 자식 노드가 있으면 재귀 호출로 출력합니다.
    # If the tag has no value and has child nodes, call the recursive function.
    else:
        result0 += "\n"
        for _child in elem:
            result0 += beautify(_child, indent + 1)
        result0 += f"{'    ' * indent}</{elem.tag}>\n"
    return result0


def labelinlist(screen_id, name):
    """
    clay_csv에 따라서 node를 분류합니다. 
    Filter nodes according to clay_csv
    """
    global resultdic

    for label in resultdic[screen_id]:
        if(label[0] == name):
            return True
    return False


def replace_label(screen_id, node):
    """
    label의 번호를 리턴합니다. 
    Returns the label number.
    """
    global resultdic
    global labeldict
    for label in resultdic[screen_id]:
        if(label[0] == node):
            label_map = int(label[1])
            return labeldict[label_map]
    return False


def recursive(screen_id, child, result_out):
    """
    원하는 역할을 하기 위해서 재귀호출을 할 수 있는 함수를 생성합니다.
    Create a function that makes a recursive call.
    """
    obj = Element("object")

        # Set bounds
    bounds = child['bounds']

        # Set difficult
    SubElement(obj, "difficult").text = '0'

    # Set bndbox
    bndbox = SubElement(obj, "bndbox")
    xmin = bounds[0]
    ymin = bounds[1]
    xmax = bounds[2]
    ymax = bounds[3]

    SubElement(bndbox, "xmin").text = str(round(xmin/4))
    SubElement(bndbox, "ymin").text = str(round(ymin/4))
    SubElement(bndbox, "xmax").text = str(round(xmax/4))
    SubElement(bndbox, "ymax").text = str(round(ymax/4))

    name = child['pointer']
    if(labelinlist(screen_id, name)):
        label_map = replace_label(screen_id, name)

        # Set name
        SubElement(obj, "name").text = label_map
        result_out.append(beautify(obj))

    # 생성한 object 태그를 문자열로 변환해서 추가합니다.
    # Convert the created object tag to a string and add it.
    if 'children' not in child:
        return

    # 자식 노드가 있는 경우 자식 노드에 대해 재귀 호출을 수행합니다.
    # If there is a child node, make a recursive call to the child node.
    for ch in child.get('children', []):
        if(ch != None):
            recursive(screen_id, ch, result_out)
        else:
            print(screen_id)
            print(ch,sep="\n")
            

def json2xml(screen_id, infile, outfile):
    """
    json2xml function
    param infile :
    ourfile :
    """

    result_out = []
    imgName = infile.replace("./combined/", "")
    imgName = imgName.replace("json", "jpg")

    # Read the file
    with open(infile, "r", encoding="UTF-8") as f:
        data = json.load(f)

    # 자식 노드에 대해 재귀 호출을 수행합니다.
    # Make a recursive call on child nodes.
    for child in data['activity']['root'].get('children', []):
        recursive(screen_id, child, result_out)

    # 그리고 해당 결과를 파일로 저장합니다.
    # And save the result to the XML file.
    with open(outfile, "w", encoding="UTF-8") as f:
        f.write("".join("<annotation><folder />"))
        f.write("".join("<filename>" + imgName + "</filename>\n"
                        "<path>" + imgName + "</path>\n" +
                        "<source><database>RICO</database></source><size><width>360</width><height>640</height><depth>3</depth></size><segmented>0</segmented>"))
        f.write("".join(result_out))
        f.write("".join("</annotation>"))


def readcsv(filename):
    """
    {screen_id:[[name0,id0]...],}
    """
    result = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')

        for row in reader:
            result.append(row)

            # print(row)
    del result[0]

    resultdic = {}
    for row in result:
        if row[0] in resultdic.keys():
            resultdic[row[0]].append([row[1], row[2]])
        else:
            resultdic[row[0]] = [[row[1], row[2]]]

    return resultdic


def readtxt(filename):
    """
    [{num}][{label}]
    """
    result = {}
    with open(filename, newline='') as f:
        while True:
            line = f.readline()
            if(not line):
                break
            num, label = line.split(':')
            num = int(num)
            label = label.strip()
            # print(f'[{num}][{label}]')
            result[num] = label
    return result


def main():
    """
    메인 함수
    The main function 
    """
    global resultdic
    global labeldict
    # print(files)

    resultdic = readcsv(clay_labels_csv)
    labeldict = readtxt(clay_label_map)

    data_path = './xml/'
    if not os.path.exists(data_path):
        os.mkdir(data_path)

    files = os.listdir(rico_dataset)
    print(len(files))
    counter = 0
    for infile in files:
        screen_id = infile.replace('.json', '').strip()
        selected_screen = resultdic.get(screen_id)
        #print(selected_screen)
        if selected_screen == None:
            print(screen_id+" is not exist!")   
        
        else:
            counter+=1
            infile_b = os.path.join(rico_dataset, f'{infile}')
            outfile = os.path.join(os.getcwd(), "xml", infile.replace("json", "xml")) 
            #print(infile, outfile)
            json2xml(screen_id, infile_b, outfile)
    print(counter)

if __name__ == "__main__":
    main()

132522
0.jpg is not exist!
0 is not exist!
1.jpg is not exist!
1 is not exist!
10.jpg is not exist!
100.jpg is not exist!
1000.jpg is not exist!
10000.jpg is not exist!
10002.jpg is not exist!
10003.jpg is not exist!
10003 is not exist!
10004.jpg is not exist!
10005.jpg is not exist!
10008.jpg is not exist!
10008 is not exist!
1001.jpg is not exist!
10010.jpg is not exist!
10011.jpg is not exist!
10012.jpg is not exist!
10013.jpg is not exist!
10014.jpg is not exist!
10015.jpg is not exist!
10016.jpg is not exist!
10017.jpg is not exist!
10018.jpg is not exist!
10019.jpg is not exist!
10019 is not exist!
1002.jpg is not exist!
10020.jpg is not exist!
10021.jpg is not exist!
10022.jpg is not exist!
10023.jpg is not exist!
10024.jpg is not exist!
10025.jpg is not exist!
10026.jpg is not exist!
10027.jpg is not exist!
10028.jpg is not exist!
10029.jpg is not exist!
1003.jpg is not exist!
10030.jpg is not exist!
10031.jpg is not exist!
10032.jpg is not exist!
10033.jpg is not exist!
10034.

10307.jpg is not exist!
10308.jpg is not exist!
10309.jpg is not exist!
1031.jpg is not exist!
10310.jpg is not exist!
10310 is not exist!
10311.jpg is not exist!
10311 is not exist!
10312.jpg is not exist!
10313.jpg is not exist!
10314.jpg is not exist!
10314 is not exist!
10315.jpg is not exist!
10316.jpg is not exist!
10317.jpg is not exist!
10318.jpg is not exist!
10318 is not exist!
10319.jpg is not exist!
1032.jpg is not exist!
10320.jpg is not exist!
1033.jpg is not exist!
1034.jpg is not exist!
10341.jpg is not exist!
10341 is not exist!
10342.jpg is not exist!
10343.jpg is not exist!
10344.jpg is not exist!
10345.jpg is not exist!
10345 is not exist!
10346.jpg is not exist!
10347.jpg is not exist!
10347 is not exist!
10348.jpg is not exist!
10349.jpg is not exist!
10349 is not exist!
1035.jpg is not exist!
10350.jpg is not exist!
10351.jpg is not exist!
10352.jpg is not exist!
10353.jpg is not exist!
10354.jpg is not exist!
10355.jpg is not exist!
10356.jpg is not exist!
10357

10677.jpg is not exist!
10678.jpg is not exist!
10679.jpg is not exist!
1068.jpg is not exist!
10680.jpg is not exist!
10681.jpg is not exist!
10683.jpg is not exist!
10684.jpg is not exist!
10685.jpg is not exist!
10686.jpg is not exist!
10687.jpg is not exist!
10688.jpg is not exist!
10689.jpg is not exist!
1069.jpg is not exist!
10690.jpg is not exist!
10691.jpg is not exist!
10692.jpg is not exist!
10693.jpg is not exist!
10696.jpg is not exist!
10697.jpg is not exist!
10697 is not exist!
107.jpg is not exist!
107 is not exist!
1070.jpg is not exist!
10701.jpg is not exist!
10701 is not exist!
10702.jpg is not exist!
10703.jpg is not exist!
10704.jpg is not exist!
10705.jpg is not exist!
10706.jpg is not exist!
10707.jpg is not exist!
10708.jpg is not exist!
10709.jpg is not exist!
10709 is not exist!
1071.jpg is not exist!
10710.jpg is not exist!
10710 is not exist!
10711.jpg is not exist!
10711 is not exist!
10712.jpg is not exist!
10713.jpg is not exist!
10714.jpg is not exist!


1100.jpg is not exist!
11000.jpg is not exist!
11001.jpg is not exist!
11002.jpg is not exist!
11003.jpg is not exist!
11004.jpg is not exist!
11005.jpg is not exist!
11006.jpg is not exist!
11007.jpg is not exist!
11008.jpg is not exist!
11009.jpg is not exist!
1101.jpg is not exist!
11010.jpg is not exist!
11010 is not exist!
11011.jpg is not exist!
11013.jpg is not exist!
11013 is not exist!
1102.jpg is not exist!
11020.jpg is not exist!
11021.jpg is not exist!
11021 is not exist!
11022.jpg is not exist!
11023.jpg is not exist!
11024.jpg is not exist!
11024 is not exist!
11025.jpg is not exist!
11026.jpg is not exist!
11027.jpg is not exist!
11028.jpg is not exist!
11029.jpg is not exist!
1103.jpg is not exist!
11030.jpg is not exist!
11031.jpg is not exist!
11032.jpg is not exist!
11032 is not exist!
11033.jpg is not exist!
11034.jpg is not exist!
11035.jpg is not exist!
11036.jpg is not exist!
11037.jpg is not exist!
11038.jpg is not exist!
11039.jpg is not exist!
1104.jpg is not 

11325.jpg is not exist!
11326.jpg is not exist!
11327.jpg is not exist!
11328.jpg is not exist!
11329.jpg is not exist!
1133.jpg is not exist!
11330.jpg is not exist!
11331.jpg is not exist!
11332.jpg is not exist!
11333.jpg is not exist!
11334.jpg is not exist!
11334 is not exist!
11335.jpg is not exist!
11336.jpg is not exist!
11337.jpg is not exist!
11338.jpg is not exist!
11339.jpg is not exist!
1134.jpg is not exist!
11340.jpg is not exist!
11341.jpg is not exist!
11342.jpg is not exist!
11343.jpg is not exist!
11344.jpg is not exist!
11345.jpg is not exist!
11346.jpg is not exist!
11347.jpg is not exist!
11348.jpg is not exist!
11349.jpg is not exist!
1135.jpg is not exist!
11350.jpg is not exist!
11350 is not exist!
11351.jpg is not exist!
11352.jpg is not exist!
11353.jpg is not exist!
11354.jpg is not exist!
11355.jpg is not exist!
11356.jpg is not exist!
11357.jpg is not exist!
11358.jpg is not exist!
11359.jpg is not exist!
1136.jpg is not exist!
11360.jpg is not exist!
1136

...

8487.jpg is not exist!
8488.jpg is not exist!
8489.jpg is not exist!
849.jpg is not exist!
8490.jpg is not exist!
8491.jpg is not exist!
8492.jpg is not exist!
8493.jpg is not exist!
8494.jpg is not exist!
8495.jpg is not exist!
8496.jpg is not exist!
8497.jpg is not exist!
8498.jpg is not exist!
8499.jpg is not exist!
85.jpg is not exist!
850.jpg is not exist!
8500.jpg is not exist!
8501.jpg is not exist!
8502.jpg is not exist!
8503.jpg is not exist!
8504.jpg is not exist!
8505.jpg is not exist!
8506.jpg is not exist!
8506 is not exist!
8507.jpg is not exist!
8508.jpg is not exist!
8509.jpg is not exist!
851.jpg is not exist!
8510.jpg is not exist!
8511.jpg is not exist!
8512.jpg is not exist!
8513.jpg is not exist!
8514.jpg is not exist!
8515.jpg is not exist!
8516.jpg is not exist!
8517.jpg is not exist!
8518.jpg is not exist!
8519.jpg is not exist!
852.jpg is not exist!
8520.jpg is not exist!
8521.jpg is not exist!
8522.jpg is not exist!
8523.jpg is not exist!
8524.jpg is not exist

8846 is not exist!
8847.jpg is not exist!
8848.jpg is not exist!
8848 is not exist!
8849.jpg is not exist!
885.jpg is not exist!
8850.jpg is not exist!
8851.jpg is not exist!
8852.jpg is not exist!
8853.jpg is not exist!
8854.jpg is not exist!
8855.jpg is not exist!
8856.jpg is not exist!
8856 is not exist!
8857.jpg is not exist!
8858.jpg is not exist!
8859.jpg is not exist!
886.jpg is not exist!
8860.jpg is not exist!
8861.jpg is not exist!
8861 is not exist!
8862.jpg is not exist!
8863.jpg is not exist!
8864.jpg is not exist!
8865.jpg is not exist!
8866.jpg is not exist!
8867.jpg is not exist!
8868.jpg is not exist!
8869.jpg is not exist!
887.jpg is not exist!
8870.jpg is not exist!
8871.jpg is not exist!
8872.jpg is not exist!
8873.jpg is not exist!
8874.jpg is not exist!
8875.jpg is not exist!
8876.jpg is not exist!
8877.jpg is not exist!
8878.jpg is not exist!
8878 is not exist!
8879.jpg is not exist!
888.jpg is not exist!
8880.jpg is not exist!
8881.jpg is not exist!
8882.jpg is 

9154.jpg is not exist!
9155.jpg is not exist!
9156.jpg is not exist!
9157.jpg is not exist!
9158.jpg is not exist!
9159.jpg is not exist!
916.jpg is not exist!
9160.jpg is not exist!
9161.jpg is not exist!
9162.jpg is not exist!
9163.jpg is not exist!
9164.jpg is not exist!
9165.jpg is not exist!
9166.jpg is not exist!
9167.jpg is not exist!
9168.jpg is not exist!
9169.jpg is not exist!
9169 is not exist!
917.jpg is not exist!
9170.jpg is not exist!
9171.jpg is not exist!
9172.jpg is not exist!
9173.jpg is not exist!
9174.jpg is not exist!
9175.jpg is not exist!
9176.jpg is not exist!
9177.jpg is not exist!
9178.jpg is not exist!
9179.jpg is not exist!
918.jpg is not exist!
9180.jpg is not exist!
9181.jpg is not exist!
9182.jpg is not exist!
9183.jpg is not exist!
9184.jpg is not exist!
9185.jpg is not exist!
9186.jpg is not exist!
9187.jpg is not exist!
9188.jpg is not exist!
9188 is not exist!
9189.jpg is not exist!
919.jpg is not exist!
9190.jpg is not exist!
9191.jpg is not exist!


9464.jpg is not exist!
9465.jpg is not exist!
9466.jpg is not exist!
9467.jpg is not exist!
9468.jpg is not exist!
9469.jpg is not exist!
947.jpg is not exist!
9470.jpg is not exist!
9471.jpg is not exist!
9472.jpg is not exist!
9474.jpg is not exist!
9474 is not exist!
9475.jpg is not exist!
9476.jpg is not exist!
9477.jpg is not exist!
9478.jpg is not exist!
9479.jpg is not exist!
948.jpg is not exist!
9480.jpg is not exist!
9481.jpg is not exist!
9481 is not exist!
9482.jpg is not exist!
9483.jpg is not exist!
9484.jpg is not exist!
9484 is not exist!
9485.jpg is not exist!
9485 is not exist!
9487.jpg is not exist!
9487 is not exist!
949.jpg is not exist!
9494.jpg is not exist!
9495.jpg is not exist!
9496.jpg is not exist!
9497.jpg is not exist!
9497 is not exist!
9498.jpg is not exist!
9499.jpg is not exist!
95.jpg is not exist!
950.jpg is not exist!
9500.jpg is not exist!
9501.jpg is not exist!
9502.jpg is not exist!
9503.jpg is not exist!
9504.jpg is not exist!
9505.jpg is not ex

9790.jpg is not exist!
9790 is not exist!
9791.jpg is not exist!
9793.jpg is not exist!
9796.jpg is not exist!
9796 is not exist!
9797.jpg is not exist!
9798.jpg is not exist!
9799.jpg is not exist!
98.jpg is not exist!
980.jpg is not exist!
9800.jpg is not exist!
9801.jpg is not exist!
9802.jpg is not exist!
9803.jpg is not exist!
9804.jpg is not exist!
9805.jpg is not exist!
9805 is not exist!
9806.jpg is not exist!
9807.jpg is not exist!
9808.jpg is not exist!
9809.jpg is not exist!
981.jpg is not exist!
9810.jpg is not exist!
9811.jpg is not exist!
9811 is not exist!
9812.jpg is not exist!
9813.jpg is not exist!
9814.jpg is not exist!
9815.jpg is not exist!
9816.jpg is not exist!
9817.jpg is not exist!
9817 is not exist!
9818.jpg is not exist!
9818 is not exist!
9819.jpg is not exist!
982.jpg is not exist!
9820.jpg is not exist!
9821.jpg is not exist!
9822.jpg is not exist!
9822 is not exist!
9823.jpg is not exist!
9824.jpg is not exist!
9825.jpg is not exist!
9826.jpg is not exist

### Train an autoencoder to learn Layout Embeddings
We can train an autoencoder to learn an embedding for UI layouts, and use it to annotate each UI with a vector representation encoding visual layout. This vector representation can be used to compute structurally similar UIs.

**Autoencoder**: Neural network consisting of two models: an encoder and a decoder to support *unsupervised* learning of lower-dimensional representations. Both models are trained together with a loss function based on the differences between inputs and
their reconstructions. Once an autoencoder is trained, the encoder portion is used to produce lower-dimensional representations of the input vectors.

***To create training inputs for the autoencoder that embed layout information, we constructed a new image for each UI encoding the bounding box regions of all leaf elements in its view hierarchy, differentiating between text and non-text elements (Figure 11).***

Since we have a big dataset, we can separate it into $90\%$ training and $10\%$ validation. 

## 2. Popular App's Screenshots

In [32]:
popular_screen_df = pd.read_excel(os.path.join(os.getcwd(), "popular_info.xlsx")) 

In [33]:
popular_screen_df

,Unnamed: 0,ui_id,installs,ratings,score,category
0,0,172,50000000,1573115,4.479546,Weather
1,1,173,50000000,1573110,4.479546,Weather
2,2,174,50000000,1573110,4.479546,Weather
3,3,175,50000000,1573110,4.479546,Weather
4,4,176,50000000,1573110,4.479546,Weather
...,...,...,...,...,...,...
2017,2017,54432,50000000,1633660,3.534384,Lifestyle
2018,2018,54433,50000000,1633660,3.534384,Lifestyle
2019,2019,54434,50000000,1633660,3.534384,Lifestyle
2020,2020,54435,50000000,1633660,3.534384,Lifestyle


## 3. RICO UI Layout Vectors
We get the RICO dataset layout vectors.

In [ ]:
lay_embs_unord = np.load(layout_embedding_csv)
lay_emb = np.zeros((len(found), 64), dtype=int)

f = open(ui_names_csv)
lay_index = json.load(f)["ui_names"]
j=0
for i in found:
    try:
        k = lay_index.index(str(i)+".png")
        i_emb = lay_embs_unord[k,:]
    except ValueError:
        i_emb = np.zeros((64,), dtype=int)
    lay_emb[j,:] = i_emb
    j+=1
    

f.close()
print(lay_emb.shape)